In [1]:
import pandas as pd
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pickle
import joblib
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
import tensorflow as tf

Class and Class ID :-
1. Asthma
2. Bronchiectasis
3. Bronchiolitis
4. Copd
5. Healthy
6. LRTI
7. Pneumonia
8. URTI

In [2]:
df=pd.read_csv('Meta_Data_of_audio_files.csv')
df=df.drop(0)
display(df)
df_feat=df
file=df_feat['File_Path']
classid=df_feat['Class_ID']
df_feat

,Unnamed: 0,File_Path,Class_ID,Class
1,1,Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav,4,Copd
2,2,Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav,4,Copd
3,3,Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav,4,Copd
4,4,Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav,4,Copd
5,5,Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav,4,Copd
...,...,...,...,...
883,883,Audio files/URTI/190_1b1_Tc_sc_Meditron.wav,8,URTI
884,884,Audio files/URTI/197_1b1_Al_sc_Meditron.wav,8,URTI
885,885,Audio files/URTI/197_1b1_Tc_sc_Meditron.wav,8,URTI
886,886,Audio files/URTI/210_1b1_Al_sc_Meditron.wav,8,URTI


,Unnamed: 0,File_Path,Class_ID,Class
1,1,Audio files/Copd/104_1b1_Ar_sc_Litt3200.wav,4,Copd
2,2,Audio files/Copd/104_1b1_Ll_sc_Litt3200.wav,4,Copd
3,3,Audio files/Copd/104_1b1_Lr_sc_Litt3200.wav,4,Copd
4,4,Audio files/Copd/104_1b1_Pl_sc_Litt3200.wav,4,Copd
5,5,Audio files/Copd/104_1b1_Pr_sc_Litt3200.wav,4,Copd
...,...,...,...,...
883,883,Audio files/URTI/190_1b1_Tc_sc_Meditron.wav,8,URTI
884,884,Audio files/URTI/197_1b1_Al_sc_Meditron.wav,8,URTI
885,885,Audio files/URTI/197_1b1_Tc_sc_Meditron.wav,8,URTI
886,886,Audio files/URTI/210_1b1_Al_sc_Meditron.wav,8,URTI


In [3]:
#for padding data
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [4]:
def generate_features(y_cut):
    sr=28000
    max_size=4800 #my max audio file feature width
    stft = padding(np.abs(librosa.stft(y_cut, n_fft=255, hop_length = 512)), 128, max_size)
    MFCCs = padding(librosa.feature.mfcc(y_cut, n_fft=255, hop_length=512,n_mfcc=128),128,max_size)
    spec_centroid = librosa.feature.spectral_centroid(y=y_cut, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y_cut, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y_cut, sr=sr)
    #Now the padding part
    image = np.array([padding(normalize(spec_bw),1, max_size)]).reshape(1,max_size)
    image = np.append(image,padding(normalize(spec_centroid),1, max_size), axis=0) 
    #repeat the padded spec_bw,spec_centroid and chroma stft until they are stft and MFCC-sized
    for i in range(0,9):
        image = np.append(image,padding(normalize(spec_bw),1, max_size), axis=0)
        image = np.append(image, padding(normalize(spec_centroid),1, max_size), axis=0)
        image = np.append(image, padding(normalize(chroma_stft),12, max_size), axis=0)
    image=np.dstack((image,np.abs(stft)))
    image=np.dstack((image,MFCCs))
    return image

In [15]:
# def get_features(df_in):   
#     features=[]     
#     labels = [] #empty array to store labels     
#     #For each species, determine how many augmentations are needed
#     df_in=df_in.reset_index()     
#     for i in df_in.Class_ID.unique():
#            print('Class_ID:',i)    
#            #all the file indices with the same species_id     
#            filelist = df_in.loc[df_in.Class_ID == i].index         
#            #filelist = df_in.loc[df_in.Class_ID == 3].index
#            for j in range(0,len(filelist)):             
#                         filename = df_in.iloc[filelist[j]].File_Path #get the filename   
#                         #define the beginning time of the signal          
#                         #tstart = df_in.iloc[filelist[j]].t_min             
#                         #tend = df_in.iloc[filelist[j]].t_max #end of signal
#                         #recording_id = df_in.iloc[filelist[j]].recording_id
#                         Class_ID = i
#                         #songtype_id = df_in.iloc[filelist[j]].songtype_id   
#                         #Load the file
#                         ys, sr = librosa.load(filename,sr=28000)  
#                         print(filename + ' '+ 'going on')
#                         print(str(j)+' out of '+ str(len(filelist))+'\n')
#                         #cut the file to signal start and end  
#                         #y_cut=y[int(round(tstart*sr)):int(round(tend*sr))]  
#                         #generate features & output numpy array          
#                         data = generate_features(ys) 
#                         features.append(data[np.newaxis,...])    
#                         labels.append(Class_ID)     
#     output=np.concatenate(features,axis=0)     
#     return(np.array(output), labels)

In [15]:
def get_features(df_in):   
    features=[]     
    labels = [] #empty array to store labels     
    #For each species, determine how many augmentations are needed
    df_in=df_in.reset_index()     
    for i in [8]:#df_in.Class_ID.unique():
           print('Class_ID:',i)    
           #all the file indices with the same species_id     
           filelist = df_in.loc[df_in.Class_ID == i].index         
           #filelist = df_in.loc[df_in.Class_ID == 3].index
           for j in range(0,len(filelist)):             
                        filename = df_in.iloc[filelist[j]].File_Path #get the filename   
                        #define the beginning time of the signal          
                        #tstart = df_in.iloc[filelist[j]].t_min             
                        #tend = df_in.iloc[filelist[j]].t_max #end of signal
                        #recording_id = df_in.iloc[filelist[j]].recording_id
                        Class_ID = i
                        #songtype_id = df_in.iloc[filelist[j]].songtype_id   
                        #Load the file
                        ys, sr = librosa.load(filename,sr=28000)  
                        print(filename + ' '+ 'going on')
                        print(str(j)+' out of '+ str(len(filelist))+'\n')
                        #cut the file to signal start and end  
                        #y_cut=y[int(round(tstart*sr)):int(round(tend*sr))]  
                        #generate features & output numpy array          
                        data = generate_features(ys) 
                        features.append(data[np.newaxis,...])    
                        labels.append(Class_ID)     
    X=np.concatenate(features,axis=0)     
    y=labels
    return 0

In [16]:
X=[]
y=[]
get_features(pd.concat([file, classid], axis=1), X, y)

Class_ID: 8
Audio files/URTI/101_1b1_Al_sc_Meditron.wav going on
0 out of 23

Audio files/URTI/101_1b1_Pr_sc_Meditron.wav going on
1 out of 23

Audio files/URTI/105_1b1_Tc_sc_Meditron.wav going on
2 out of 23

Audio files/URTI/119_1b1_Ar_sc_Meditron.wav going on
3 out of 23

Audio files/URTI/129_1b1_Ar_sc_Meditron.wav going on
4 out of 23

Audio files/URTI/131_1b1_Al_sc_Meditron.wav going on
5 out of 23

Audio files/URTI/137_1b1_Ar_sc_Meditron.wav going on
6 out of 23

Audio files/URTI/137_1b1_Ll_sc_Meditron.wav going on
7 out of 23

Audio files/URTI/148_1b1_Al_sc_Meditron.wav going on
8 out of 23

Audio files/URTI/150_1b2_Al_sc_Meditron.wav going on
9 out of 23

Audio files/URTI/164_1b1_Ll_sc_Meditron.wav going on
10 out of 23

Audio files/URTI/165_1b1_Ar_sc_Meditron.wav going on
11 out of 23

Audio files/URTI/165_1b1_Pl_sc_Meditron.wav going on
12 out of 23

Audio files/URTI/165_1b1_Pr_sc_Meditron.wav going on
13 out of 23

Audio files/URTI/188_1b1_Al_sc_Meditron.wav going on
14 out 

0

In [17]:
X


[]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123, stratify=y)
print(X_train.shape,X_test.shape)

In [ ]:
X_train = np.array((X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train)))
X_test = np.array((X_test-np.min(X_test))/(np.max(X_test)-np.min(X_test)))
X_train = X_train/np.std(X_train)
X_test = X_test/np.std(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)
print(X_train.shape, X_test.shape, X_val.shape, len(y_train), len(y_test), len(y_val))

<Font size = 6> <b> CNN Model </b></font>

In [22]:
input_shape=(128,1000,3)
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(64, activation='relu'))
CNNmodel.add(layers.Dropout(0.2))
CNNmodel.add(layers.Dense(32, activation='relu'))
CNNmodel.add(layers.Dense(24, activation='softmax'))

In [ ]:
history = CNNmodel.fit(X_train, y_train, epochs=20, validation_data= (X_val, y_val))

<b>For preventing overfit !!!!!</b>

In [ ]:
input_shape=(128,1000,3)
CNNmodel = models.Sequential()
CNNmodel.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
CNNmodel.add(layers.MaxPooling2D((2, 2)))
CNNmodel.add(layers.Flatten())
CNNmodel.add(layers.Dense(32, activation='relu'))
CNNmodel.add(layers.Dense(24, activation='softmax'))
CNNmodel.summary()

<b> For plotting</b>

In [ ]:
history_dict=history.history
loss_values=history_dict['loss']
acc_values=history_dict['accuracy']
val_loss_values = history_dict['val_loss']
val_acc_values=history_dict['val_accuracy']
epochs=range(1,21)
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(15,5))
ax1.plot(epochs,loss_values,'bo',label='Training Loss')
ax1.plot(epochs,val_loss_values,'orange', label='Validation Loss')
ax1.set_title('Training and validation loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(epochs,acc_values,'bo', label='Training accuracy')
ax2.plot(epochs,val_acc_values,'orange',label='Validation accuracy')
ax2.set_title('Training and validation accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.show()